We want to find the integral over the whole real number line of the magnitude-squared response of a Butterworth filter of N-th order. So, we want to evaluate:
$$ \int_{-\infty}^{\infty} \frac{1}{1 + x^{2N}} dx$$
This integral equals the total energy of the impulse response of the filter (by Parseval's theorem - verify that). The idea is that different orders may have different energies and we may want to normalize the output energy by either amplifying the output as function of $N$ (thereby raising the "height" of the passband) or by raising the cutoff frequency (thereby increasing the width of the passband).

In [1]:
# First order:
f1(x) = 1 / (1 + x^2)
integral(f1(x), x, -oo, oo)

pi

In [2]:
# Second order:
f2(x) = 1 / (1 + x^4)
integral(f2(x), x, -oo, oo)

1/2*sqrt(2)*pi

In [3]:
# Third order:
f2(x) = 1 / (1 + x^6)
integral(f2(x), x, -oo, oo)

2/3*pi

In [4]:
# OK, let's try the general case:
var("N")
assume(N >= 1)
assume(N, 'integer')
fN(x) = 1 / (1 + x^(2*N))
integral(fN(x), x, -oo, oo)

pi/(N*sin(1/2*pi/N))

Now, let's see what happens to the total energy, when we apply the N-th order filter M times, corresponding to the following integral:
$$ \int_{-\infty}^{\infty} \frac{1}{(1 + x^{2N})^M} dx$$

In [5]:
var("M")
assume(M >= 1)
assume(M, 'integer')
assume(2*M*N-1 > 0)   # actually follows from N >= 1, M >= 1 but sage needs it
fNM(x) = 1 / (1 + x^(2*N))^M
integral(fNM(x), x, -oo, oo)

pi*gamma(M - 1/2/N)/(N*gamma(M)*gamma(-1/2/N + 1)*sin(1/2*pi/N))

OK - nice. Sage can evaluate these energy integrals in terms of the order N and the number of passes M with ease. The solution is:
$$\frac{\pi \Gamma\left(M - \frac{1}{2 \, N}\right)}{N \Gamma\left(M\right) \Gamma\left(-\frac{1}{2 \, N} + 1\right) \sin\left(\frac{\pi}{2 \, N}\right)}$$
...but how is such an integral evaluated? Maybe using the residue theorem? Here's and interactive plot where the user selects N and M:

In [ ]:
@interact
def butterPlot(N = slider(1, 10, 1, 1), 
               M = slider(1, 10, 1, 1),
               Normalize = checkbox(default=True)):        
    E = pi*gamma(M - 1/2/N)/(N*gamma(M)*gamma(-1/2/N + 1)*sin(1/2*pi/N))
    print "Energy:", E, "=" , n(E)
    k = 1
    if(Normalize):
        k = 1/E
    f(x) = 1 / (1 + (x/k)^(2*N))^M
    ne = integral(f(x), x, -oo, oo)
    print "normalized Energy:", ne, "=" , n(ne)
    p = plot(f(x), -2, 2)
    #p.show()

OK - this seems to work. problem solved. By the way, the latex code for the solution was also generated by sage via appyling the `latex` command to sage's output:

In [7]:
latex(pi*gamma(M - 1/2/N)/(N*gamma(M)*gamma(-1/2/N + 1)*sin(1/2*pi/N)))

\frac{\pi \Gamma\left(M - \frac{1}{2 \, N}\right)}{N \Gamma\left(M\right) \Gamma\left(-\frac{1}{2 \, N} + 1\right) \sin\left(\frac{\pi}{2 \, N}\right)}

In [8]:
# one more thing - let N be fixed at N=1 and vary only M - corresponding to applying a first orde filter M times:
f1M(x) = 1 / (1 + x^2)^M
integral(f1M(x), x, -oo, oo)

sqrt(pi)*gamma(M - 1/2)/gamma(M)

In [9]:
# and for N=2 (for example a lowpass RBJ biquad with Q=1/sqrt(1)):
f2M(x) = 1 / (1 + x^4)^M
integral(f2M(x), x, -oo, oo)

1/2*sqrt(2)*pi*gamma(M - 1/4)/(gamma(3/4)*gamma(M))

In [11]:
# The old way of compensating for higher order and multiple passes was to let the
# magnitude-squared responses all pass through (1,g) where g=1/2, leading to a
# qualitatively similar but different normalization function - let's compare both
# functions (of N and M) in a 3d plot::

def cutoffNormalizer(N, M):
    g = 1/sqrt(2)
    return (g^(-2/M)-1)^(-1/(2*N))

def energyNormalizer(N, M):
    E = pi*gamma(M - 1/2/N)/(N*gamma(M)*gamma(-1/2/N + 1)*sin(1/2*pi/N))
    return pi*1/E

var("y")
maxN = 10
maxM = 10
pp = 10  # oversample x10 to make a smoother plot
C = plot3d(cutoffNormalizer, (x,1,maxN),(y,1,maxM), opacity=0.7, color='blue', viewer="threejs", plot_points=pp)
E = plot3d(energyNormalizer, (x,1,maxN),(y,1,maxM), opacity=0.7, color='red',  viewer="threejs", plot_points=pp)
#C + E